In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Intro

Hello, everyone.
I heard you guys are called Kagglers here.

I'm a student who just got to know Kaggle.

While studying analysis using data, I chose this data because I thought it would be fun to deal with the data in my country.

I've tried to code every single process of dealing with data.
I hope it will be a good material for you.

안녕하세요 여러분!! 
여기서는 여러분을 캐글러라고 부른다면서요?? 

저는 캐글을 알게된지 얼마 안된 학생입니다. 

데이터를 분석하는 것에 관심만 가지고 있다가, 실제로 데이터의 구조를 알아보는 활동을 직접 해보고 싶어서 이렇게 EDA를 진행하게 됐습니다.

데이터를 이용한 분석을 공부하는 김에, 제 나라의 데이터를 다뤄보는 것이 재밌을 것 같아서 이 데이터를 선정하게 됐습니다. 

데이터를 다루는 과정을 하나하나 코드에 담고자 노력했으니
여러분에게 좋은 자료가 되었으면 좋겠습니다. 


## the shape of the data

### The data looks like this:

- id
- year : study conducted
- wave : from wave 1st in 2005 to wave 14th in 2018
- region: 1) Seoul 2) Kyeong-gi 3) Kyoung-nam 4) Kyoung-buk 5) Chung-nam 6) Gang-won &. Chung-buk 7) Jeolla & Jeju
- income: yearly income in M KRW(Million Korean Won. 1100 KRW = 1 USD)
- family_member: no. of family members
- gender: 1) male 2) female
- year_born
- education_level: 1) no education(under 7 yrs-old) 2) no education(7 & over 7 yrs-old) 3) elementary 4) middle school 5) high school 6) college 7) university degree 8) MA 9) doctoral degree
- marriage: marital status. 1) not applicable (under 18) 2) married 3) separated by death 4) separated 5) not married yet 6) others
- religion: 1) have religion 2) do not have
- occupation: this will be provided in separated code book
- company_size
- reasonnoneworker: 1) no capable 2) in military service 3) studying in school 4) prepare for school 5) preprare to apply job 6) house worker 7) caring kids at home 8) nursing 9) giving-up economic activities 10) no intention to work 11) others

For more information, see the following link: https://www.kaggle.com/hongsean/korea-income-and-welfare

In [ ]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# 1. Data preprocessing

Before we get started with the data, let's go and see what it looks like.

데이터를 본격적으로 다루기에 앞서서 데이터가 어떤 모양인지 불러와 보도록 하겠습니다. 

In [ ]:
file_path = '../input/korea-income-and-welfare/Korea Income and Welfare.csv'

df = pd.read_csv(file_path, sep = '\t')

print(df.shape)
print("--------------------------------")
print(df.info())

In [ ]:
df.head(10)

Looking at the appearance of the data, we can see that preprocessing is necessary to make it more manageable.  
Let's split the sentences separated by commona into separate columns.  
The following article was referenced.  
https://stackoverflow.com/questions/37600711/pandas-split-column-into-multiple-columns-by-comma

데이터의 모양을 살펴보니 좀 더 다루기 쉽게 전처리가 필요하다는 것을 알수 있습니다.  
comma 로 구분되어 있는 문장들을 쪼개서 각각의 column들로 만들어봅시다.  
다음과 같은 글을 참고했습니다.  
https://stackoverflow.com/questions/37600711/pandas-split-column-into-multiple-columns-by-comma

In [ ]:
col_name = 'id,year,wave,region,income,family_member,gender,year_born,education_level,marriage,religion,occupation,company_size,reason_none_worker'
cols = col_name.split(',')
new_df = df[col_name].str.split(",", n = 13, expand = True)
new_df.columns = cols
new_df

If you look further at the data, you'll see that there are empty data everywhere.
Fill in the blanks with NaN.

Since all types of data are string, the type of numeric data is changed to int.


데이터들을 좀 더 살펴보면 곳곳에 비어있는 데이터들이 있다는 것을 알 수 있습니다. 
빈칸을 NaN으로 채워줍니다. 

데이터들의 타입이 모두 string 이므로 숫자 데이터의 타입은 int로 바꿔줍니다.

In [ ]:
for col in cols:
    new_df[col] = pd.to_numeric(new_df[col], errors='coerce')
       
# for col in cols:
#     print(type(new_df[col][0]))
new_df.head(10)

Visualizing the loss is as follows:

결손을 시각화하면 다음과 같습니다.

In [ ]:
print(new_df.isnull().sum())
sns.heatmap(new_df.isnull(), cbar=False)

# 2. Occupation & income

Let's look at the relationship between jobs and income to see how much they earn.

First, let's find out which jobs account for the overall portion of Korean income and which jobs are high-income jobs.

직업과 수입간 관계를 통해서 어떤 직업들이 어느정도의 수입을 버는지 알아보겠습니다. 

먼저 한국 수입의 전체적인 비중을 차지하는 직업을 알아보고, 어떤 직업이 고소득 직업인지 알아보겠습니다. 

The name and classification for the job are in 'job_code_translated.xlsx'.  
Let's bring Excel file to update the job data.

직업에 대한 이름과 분류는 job_code_translated.xlsx 에 있습니다. 
직업데이터를 갱신하기 위해 엑셀파일을 불러와줍니다. 

In [ ]:
job_df = pd.read_excel('../input/korea-income-and-welfare/job_code_translated.xlsx')  
job_df.head(10)

In [ ]:
df_LEFT_JOIN = pd.merge(new_df, job_df, left_on='occupation', right_on='job_code', how='left')
#이렇게 만든 최종적인 데이터 프레임을 좀 더 깔끔하게 만들자. 
kor_df = df_LEFT_JOIN[['id', 'year', 'wave', 'region', 'income', 'family_member', 'gender', 'year_born', 'education_level', 'marriage', 'religion', 'occupation', 'company_size', 'reason_none_worker', 'job_title']]
kor_df

First of all, let's find out what occupations have a high proportion in terms of overall production activities.


각각의 직업만 놓고 봤을 때 수입이 높은 직업은 무엇이며, 전체적인 생산활동으로 봤을 때 비중이 높은 직업은 무엇인지 알아보도록 하겠습니다. 

In [ ]:
top_k_income = kor_df.groupby(['year','job_title'], sort=True, as_index = False).sum()
top_k_income = top_k_income.sort_values(by = ['year', 'income'], ascending=[True,False])
top_k_income_2015 = top_k_income[top_k_income['year'] == 2015]
top_k_income_2015 = top_k_income_2015[['job_title', 'income']].head(10)
sns.barplot(x="income", y="job_title", data=top_k_income_2015)

The data were analyzed based on 2015.  
You can see that there are many workers working in farms in Korea.  
In addition, it appears that there are many other people in the transportation business, and the proportion of people in operation and operation is also high.


데이터는 2015년을 기준으로 분석을 진행했습니다. 
한국은 농가에서 일하는 노동자가 많다는 것을 알 수 있습니다.  
그 밖에 운송업을 하는 인구도 많은 것으로 보이며, 운영과 영업을 하는 사람들의 비율 또한 높은 것을 확인할 수 있습니다. 

Simple income statistics for a job include:

직업에 대한 간단한 소득 통계는 다음과 같습니다. 

In [ ]:
income_agg=kor_df.groupby('job_title').income.agg(['mean','min','max','sum']).sort_values(by = 'mean', ascending=False)
income_agg

Here are the top 10 jobs in 2015 data that earn an average of high incomes:

2015년 데이터에서 평균적으로 많은 소득을 받는 상위  10개 직업은 다음과 같습니다. 

In [ ]:
top_k_income_job = kor_df.groupby(['year','job_title'], sort=True, as_index = False).mean()
top_k_income_job = top_k_income_job.sort_values(by = ['year', 'income'], ascending=[True,False])
top_k_income_job_2015 = top_k_income_job[top_k_income_job['year'] == 2015]
top_k_income_job_2015 = top_k_income_job_2015[['job_title', 'income']].head(10)
sns.barplot(x="income", y="job_title", data=top_k_income_job_2015)


What jobs have lower incomes than the overall average?  

전체 평균보다 수입이 낮은 직업은 어떠한 것이 있을까요.

In [ ]:
total_mean = kor_df[kor_df['occupation']!= 'NaN'].income.mean()
print(total_mean)
job_mean = kor_df.groupby('job_title')['income'].mean()
job_over_mean = job_mean[job_mean <= total_mean].sort_values(ascending=True)
print(job_over_mean)

In [ ]:
n = 10
job_list = job_over_mean.head(n).index.tolist()
job_list.append('mean')
x_pos = np.arange(n+1)
income_list = job_over_mean.head(n).tolist()
income_list.append(total_mean)

bar_list = plt.bar(x_pos, income_list, align='center', alpha=0.5)
bar_list[n].set_color('r')
plt.plot([0.,n], [total_mean, total_mean], 'k--')
plt.xticks(x_pos, job_list, rotation='vertical')

plt.ylabel('income')
plt.title('2015 job income list with mean')

plt.show()

Most of Korea's profit-making jobs were agriculture-related jobs,  
and individual profits show that they earn less than average.

한국의 대부분의 수익을 올리는 직종이 농업관련 직종이었는데,  
개별 수익으로 놓고 보면 평균 대비 적은 수익을 받는다는 것을 볼 수 있습니다. 

## Add Gender

Let's take a look at the data similar to the above and categorize it by gender.  
This time, I will use the 2017 data.

위와 비슷한 데이터를 가지고 성별로 분류해서 살펴보도록 하겠습니다. 
이번엔 2017년도 자료를 사용해 보겠습니다. 

In [ ]:
def ratio_plot_by_group(data , value , group = None,  fig_kws={"stacked" : True, "title" : ""}) :
    if group is None :
        result = data[value].value_counts(normalize=True)
        multi_index = pd.MultiIndex.from_product([[value], result.index.unique().tolist()], 
                                         names=["group", value])
        result.index = multi_index
        result = result.unstack()
    else :
        result = data.groupby(group)[value].value_counts(normalize=True).unstack()
    return result.plot(kind="bar", 
                stacked= fig_kws.get("stacked" , True), 
                title=fig_kws.get("title",""))
    
    

In [ ]:
top_k_income_job_2017 = top_k_income_job[top_k_income_job['year'] == 2017]
top_k_income_job_2017 = top_k_income_job_2017.head(10)
job_k_list = top_k_income_job_2017['job_title'].values

data = kor_df[kor_df['year']==2017]
data = data[data['job_title'].isin(job_k_list)]
ratio_plot_by_group(data ,"gender", group="job_title",fig_kws = {"stacked":True,"title" : "2017 gender ratio plot: high income"})
plt.show()

In [ ]:
low_k_income_job = kor_df.groupby(['year','job_title'], sort=True, as_index = False).mean()
low_k_income_job = low_k_income_job.sort_values(by = ['year', 'income'], ascending=[True,True])
low_k_income_job_2017 = low_k_income_job[low_k_income_job['year'] == 2017]
low_k_income_job_2017 = low_k_income_job_2017.head(10)
job_k_list = low_k_income_job_2017['job_title'].values

data = kor_df[kor_df['year']==2017]
data = data[data['job_title'].isin(job_k_list)]
ratio_plot_by_group(data ,"gender", group="job_title",fig_kws = {"stacked":True,"title" : "2017 gender ratio plot: low income"})
plt.show()

The top 10 highly paid jobs are mostly male, and the bottom 10 jobs have a slightly higher percentage of women.

There was a higher percentage of males in the work of exertion, and much higher proportion of females in cooking and household chores.

높은 임금을 받는 상위 10개 직업은 대부분 남성이라는 것을 볼 수 있으며, 하위 10개 직업은 여성의 비율이 조금 더 높은 것을 볼 수 있습니다. 

힘을 쓰는 일에는 남성의 비율이 좀 더 높고, 요리 및 가사노동은 여성의 비율이 훨씬 높았습니다. 

# 3. Education & Region

Let's take a quick look at the educational level of the people living in each region.  

각 지역에서 살고 있는 사람들의 교육수준 데이터를 간단하게 살펴봅시다.

In [ ]:
kor_df
data = kor_df[kor_df['year']==2017]
ratio_plot_by_group(data ,"region", group="education_level",fig_kws = {"stacked":True,"title" : "2017 region ratio plot: education"})
x = np.array([0,1,2,3,4,5,6,7])
region = ['Seoul', 'Kyeong-gi', 'Kyoung-nam', 'Kyoung-buk', 'Chung-nam', 'Gang-won &. Chung-buk', 'Jeolla & Jeju']
education_level = ['no education', 'elementary', 'middle school', 'high school', 'college', 'university degree', 'MA', 'doctoral degree']
plt.xticks(x, education_level)
plt.legend(labels=region, loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()

Data show that highly educated people live in the metropolitan area(Seoul, Kyeong-gi).

고학력자는 수도권에 몰려 생활하는 것을 데이터로 확인할 수 있습니다. 

# 4. Correlation : income, family, gender, year_born, eduLv(BA) 

In [ ]:
kor_df_id = kor_df.groupby(['id'], sort=True, as_index = False).mean()
ax = sns.boxplot(x=kor_df_id["year"])

In [ ]:
kor_df_id = kor_df_id[2012<=kor_df_id['year']]
kor_df_id = kor_df_id[kor_df_id['year']<=2015]
kor_df_id

Let's take a look at the percentage of data for the year and proceed analysis with comparatively recent data.

연도에 대한 데이터 비율이 어떤지 살펴보고 비교적 최근이라 할 수 있는 데이터를 가지고와 분석을 진행해 보겠습니다.

In [ ]:
kor_df_id[["family_member","gender","education_level","religion"]] = kor_df_id[["family_member","gender","education_level","religion"]].apply(lambda x: round(x))
kor_df_id["edu_BA"] = kor_df_id["education_level"].apply(lambda x : 1 if x >= 6 else 0) 
BA = kor_df_id["edu_BA"].sum()
Non_BA = len(kor_df_id["edu_BA"]) - BA
ratio = [BA, Non_BA]
labels = ['BA', 'Non_BA']

plt.pie(ratio, labels=labels, autopct='%.1f%%')
plt.show()

Overall data shows a 28 percent rate of college graduates.

전체 데이터에서는 대졸자가 28퍼센트 정도의 비율을 보입니다. 

In [ ]:
data = kor_df_id[["income","family_member","gender","year_born","education_level","marriage","religion"]]
data = (data-data.min())/(data.max()-data.min())
plt.figure(figsize=(15,15))
sns.heatmap(data = data.corr(), annot=True, 
fmt = '.2f', linewidths=.5, cmap='Blues')

In [ ]:
kor_df_id["year_born"].mean()

But I think the average age is too high.
Set the average age to 40 and proceed with the analysis again.

그런데 평균연령이 너무 높은 것 같습니다. 
평균연령을 30~40대로 맞추고 다시 분석을 진행해봅니다.

In [ ]:
kor_df_id = kor_df_id[1960<kor_df_id['year_born']]
kor_df_id = kor_df_id[kor_df_id['year_born']<1985]
kor_df_id

In [ ]:
kor_df_id[["family_member","gender","education_level","religion"]] = kor_df_id[["family_member","gender","education_level","religion"]].apply(lambda x: round(x))
kor_df_id["edu_BA"] = kor_df_id["education_level"].apply(lambda x : 1 if x >= 6 else 0) 
BA = kor_df_id["edu_BA"].sum()
Non_BA = len(kor_df_id["edu_BA"]) - BA
ratio = [BA, Non_BA]
labels = ['BA', 'Non_BA']

plt.pie(ratio, labels=labels, autopct='%.1f%%')
plt.show()

The proportion of college graduates has increased dramatically.
It seems to show the enthusiasm of study in Korea. (Or eagerness for a college diploma)

I think this is the result of college graduates showing significant correlation in the job market.

대졸자의 비율이 엄청나게 높아졌습니다. 
한국의 학구열을 보여주는 듯 합니다.  
대졸자가 취업시장에서 유의미한 상관을 보였기 때문에 생긴 결과라 생각합니다. 

In [ ]:
data = kor_df_id[["income","family_member","gender","year_born","education_level","marriage","religion"]]
data = (data-data.min())/(data.max()-data.min())
plt.figure(figsize=(15,15))
sns.heatmap(data = data.corr(), annot=True, 
fmt = '.2f', linewidths=.5, cmap='Blues')

# 5. Income comparison by gender(with. t-test)

In [ ]:
kor_df_new = kor_df[2014<kor_df['year']]
kor_df_new = kor_df_new[2020>kor_df['year']]
male = kor_df_new.loc[kor_df_new['gender']==1]
female = kor_df_new.loc[kor_df_new['gender']==2]

print(male.size)
print(female.size)

Let's learn about skewness and kurtosis to see if the data is skewed in a particular direction.

The skewness means that the probability distribution of a probability variable has asymmetry.  
A=0 means normal distribution, a<0 means right bias, a>0 means left bias.

The Kurtosis means the sharpness of the probability distribution.  
a=3 means normal distribution, a=3 means more gentle than regular distribution, and a>3 means sharper than regular distribution.


특정 방향으로 왜곡되어 있는 데이터인지 알아보기 위해 왜도와 첨도에 대해 알아봅시다. 

왜도(비대칭도)란, 확률변수의 확률분포가 비대칭성을 가지고 있음을 뜻합니다.

영어로는 Skewness라고 하며 a=0이면 정규분포, a<0이면 오른쪽으로 치우침, a>0이면 왼쪽으로 치우침을 의미합니다.

첨도란, 확률분포의 뾰족한 정도를 뜻합니다. 

영어로는 Kurtosis라고 하며 a=3이면 정규분포, a<3이면 정규분포보다 완만함, a>3이면 정규분포 보다 뾰족함을 의미합니다.

In [ ]:
numerical_feats = male.dtypes[male.dtypes != "object"].index 
for col in numerical_feats: 
    print('{:15}'.format(col), 'Skewness: {:05.2f}'.format(male[col].skew()) , 
          ' ' , 
          'Kurtosis: {:06.2f}'.format(male[col].kurt()) 
         )


In [ ]:
f, ax = plt.subplots(figsize = (10, 6)) 
sns.distplot(male["income"])
plt.xlim(-10000, 50000)
plt.title('male income')


In [ ]:
numerical_feats = female.dtypes[male.dtypes != "object"].index 
for col in numerical_feats: 
    print('{:15}'.format(col), 'Skewness: {:05.2f}'.format(female[col].skew()) , 
          ' ' , 
          'Kurtosis: {:06.2f}'.format(female[col].kurt()) 
         )


In [ ]:
f, ax = plt.subplots(figsize = (10, 6)) 
sns.distplot(female["income"])
plt.xlim(-10000, 50000)
plt.title('female income')

Let's see the results after normalizing it.

이를 정규화 시켜준 뒤 결과를 보도록 하겠습니다. 

In [ ]:
male["Log_income"] = np.log1p(male["income"]) 
f, ax = plt.subplots(figsize = (10, 6)) 
plt.title('male income _ norm')
sns.distplot(male["Log_income"]) 
print("Skewness: {:.3f}".format(male["Log_income"].skew())) 
print("Kurtosis: {:.3f}".format(male["Log_income"].kurt()))

In [ ]:
female["Log_income"] = np.log1p(female["income"]) 
f, ax = plt.subplots(figsize = (10, 6)) 
plt.title('female income _ norm')
sns.distplot(female["Log_income"]) 
print("Skewness: {:.3f}".format(female["Log_income"].skew())) 
print("Kurtosis: {:.3f}".format(female["Log_income"].kurt()))



If you look at the degree of skewness of each regularization function, you can see that the male group is skewed to the right and the female group is skewed to the left.


각각의 정규화 함수가 치우쳐진 정도를 확인해보면 남성그룹은 오른쪽으로 여성그룹은 왼쪽으로 치우쳐진 것을 확인해볼 수 있습니다. 

Python uses a library called scipy to test the t-test between two groups.
You can see the results of the test by putting two groups of series data into the ttest() function.
Equal-var, the parameter of this function, is to choose one of two methods of t-test.
The first is to pretend that the variances between the two groups are equal, and the second is to pretend that the variances are not equal.

Assuming that the means for the male and female populations are the same, you can see that the p-value is zero if you proceed with the t-test. This indicates that there is a significant difference between the two groups.


파이썬에서는 scipy라는 라이브러리를 활용해서 두 집단 간의 t-test를 검정할 수 있습니다.
ttest() 함수에 두 집단의 시리즈 데이터를 넣는 것으로 검정의 결과를 확인할 수 있는데,  
이 함수의 파라미터인 equal-var는 t-test의 두 가지 방법중에 하나를 선택하는 것입니다.  
첫 번째는 두 집단간의 분산이 같은경우, 그리고 두번째는 두 집단의 분산이 같지 않은 경우를 가장한 것입니다. 

남성집단과 여성집단의 평균이 같다고 가정하고 t-test를 진행하면 p-value가 0이 나오는 것을 확인할 수 있습니다. 이는 두 집단이 유의미한 차이가 있다는 것을 나타냅니다.  

In [ ]:
tTestResult = stats.ttest_ind(male['Log_income'].dropna(),female['Log_income'].dropna())
tTestResultDiffVar = stats.ttest_ind(male['Log_income'].dropna(),female['Log_income'].dropna(), equal_var = False)
print("The t-statistic and p-value assuming equal variances is %.3f and %.3f." % tTestResult)
print("The t-statistic and p-value not assuming equal variances is %.3f and %.3f." % tTestResultDiffVar)